# CIS 9
# Unsupervised Learning: k-Means Clustering 

Reading

Python Data Handbook Chapter 5: k-Means Clustering
- Introducing k-Means
- k-Means Algorithm: Expectation–Maximization

### Clustering

In unsupervised learning, there are input features but no output labels to "learn" from. In other words, there are no output labels that act as a supervisor to tell us how accurate the predicted output is.  Instead, the goal of unsupervised learning is to find hidden patterns in the unlabeled data. Unsupervised learning is learning by observation, rather than learning by examples as in supervised learning.

A common way to find hidden patterns in unlabeled data is to group the data into clusters based on the similarities in the data. A cluster is a grouping of data that are similar to each other in some way, and are dissimilar to data belonging to other clusters.

There are multiple ways to cluster data, k-means is one of the most straight forward ways. K-means is a _partition based clustering_, which means data are partitioned into k mutually exclusive clusters, where k is a user defined value. Other types of clustering are _hierarchical clustering_ (grouping data with similarity together until there are a certain number of clusters), and _density based clustering_ (plot data and group the ones that are closer together into one cluster).

### k-Means Clustering

The k-means algorithm is an iterative algorithm. It starts by randomly creating k _centroids_ or centers of a cluster. Then it puts each data point into a cluster based on how close the data value is to a particular centroid. When all data are in clusters, the algorithm calculates the mean of each cluster, and the mean serves as a new centroid for a new cluster. The algorithm keeps repeating the 2 steps of putting all data into the new clusters and calculates the new centroids, and this iterative process ends when all the newly calculated centroids are the same value as the previous centroid.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
from sklearn.cluster import KMeans

We start with data of customers at a mall. The data has customers' ID, genre, age, income, and spending score.  The spending score is important to the mall because it shows how often the customers shops / spends money at the mall. We want to see if there's customer attribute that's associated with a higher spending score.
The input data is _mall.csv_ ([source](https://www.kaggle.com/shwetabh123/mall-customers)).

In [3]:
# 1. Read in data from mall.csv. Show the size of the data and the first lines of data.
data = pd.read_csv('mall.csv')
print(data.shape)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'mall.csv'

In [ ]:
# 2. Find basic statistics on the data (mean, standard devision, min, max, quartile of each feature or column)
data.describe()

Since we're interested in the general attribute of customers, create a new DataFrame called testdata that's easier to work with. 

In [ ]:
# 3. a. remove the CustomerID field
#    b. simplify the column labels to genre, age, income, and spend
#    c. print the first few lines of the data to show the change
testdata = data.drop(columns=['CustomerID'])
testdata.columns = ['genre','age','income','spend']
testdata.head()

In [ ]:
# 4. To get some preliminary idea of the customer attributes, first let's see the ratio of male / female shoppers
# Show a bar chart of the number of male and female shoppers.
#import collections
plt.style.use('seaborn-whitegrid')

plotD = testdata.groupby("genre").age.count()
print(plotD)
plt.bar(plotD.index, plotD)

In [ ]:
# Other way to find the number of males and females:

female = np.sum(testdata.genre == "Female")
male = np.sum(testdata.genre == "Male")
plt.bar(("Male", "Female"), (male, female))

In [ ]:
# 5. Next, let's view the distribution of the age, income, and spending score
# Are the shoppers generally younger or older? Low, mid, or high income? 
# Is the spending score a normal or Gaussian distribution?
# What's significant about the spending score being a Gaussian distribution?

fig = plt.figure(figsize=(12,4))
plt.subplot(1,3,1)   # 1 row, 3 cols, subplot 1
plt.subplots_adjust(hspace=0.4,wspace=0.4)
plt.hist(testdata['age'])
plt.xlabel("Age")
plt.subplot(1,3,2)   # 1 row, 3 cols, subplot 2
plt.hist(testdata['income'])
plt.xlabel("Income")
plt.subplot(1,3,3)   # 1 row, 3 cols, subplot 3
plt.hist(testdata['spend'])
plt.xlabel("Spending")

There are more shoppers in the lower half of the age distribution, so the shoppers are younger.
<br>The majority of shoppers are in the lower and middle range of the income distribution, there are fewer high income shoppers
<br>The spending is a Gaussian distribution, which means that the data are a good sample of shopping habits at the mall.

In [ ]:
# Another way to have an overview of the data. 
# Here we see both the distribution and the correaltion of data

import seaborn as sns
sns.pairplot(testdata)

First we see if the income has any effect on the spending score

In [ ]:
# 6. 
X = testdata.drop(columns=['genre','age'])
print(X.columns.values)

To use the k-means model, we need to determine the k value that will work best with the dataset (this is similar to the knn model for supervised learning). There is no rule of thumb for determining k, but there's an _elbow method_ that works well in practice.

The basic idea of k-means clustering is to define clusters such that the distance between each data point in the cluster and the centroid is as small as possible, because it means the data in the cluster are as tight together as possible. The measurement of this distance is called the within-cluster sum of square (WCSS). 

To find the optimal k value, we try different k values with the k-means algorithm and plot the WCSS of each k value. As the k value increases, the WCSS value will drop as the cluster gets tighter, until the k value gets too large and we don't see the benefit of having more clusters. The plot typically has a bend (like a bent elbow), and that's the location of the optimal k value.

In [ ]:
# 7. Calculate the optimal k value
wcss=[]
for i in range(1,11):
    kmeans=KMeans(n_clusters=i,init='k-means++')   # k-means++ is a good general purpose seed value for randomly selecting the initial centroids
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
fig = plt.figure(figsize=(5,5))
plt.plot(np.arange(1,11),wcss)
plt.title('Elbow Curve')
plt.show() 

In [ ]:
# 8. From the above plot, we use k = 5 to create the model, train and test the model
kmeans = KMeans(n_clusters=5,init='k-means++')
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
print(y_kmeans)
# we see that there are 5 clusters in the k-means output, 
# with values from 0 to 4

Using the k-means output, we plot the income vs. spending score

In [ ]:
# 9.
fig = plt.figure(figsize=(8,5))

# plotting each cluster
plt.scatter(X[y_kmeans == 0].income, X[y_kmeans == 0].spend, label = 'Cluster 1')
plt.scatter(X[y_kmeans == 1].income, X[y_kmeans == 1].spend, label = 'Cluster 2')
plt.scatter(X[y_kmeans == 2].income, X[y_kmeans == 2].spend, label = 'Cluster 3')
plt.scatter(X[y_kmeans == 3].income, X[y_kmeans == 3].spend, label = 'Cluster 4')
plt.scatter(X[y_kmeans == 4].income, X[y_kmeans == 4].spend, label = 'Cluster 5')

# overfitting: try a higher number of clusters (higher n)
# and then un-comment the appropriate lines below to match your n value
# (up to 8) to see what the clusters look like
#plt.scatter(X[y_kmeans == 5].income, X[y_kmeans == 5].spend, label = 'Cluster 6')
#plt.scatter(X[y_kmeans == 6].income, X[y_kmeans == 6].spend, label = 'Cluster 7')
#plt.scatter(X[y_kmeans == 7].income, X[y_kmeans == 7].spend, label = 'Cluster 8')

# Note that the 5 - 8 lines above ideally should be in a loop
# They are listed one-by-one to clearly show what's being plotted

# plot the centroid
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c = 'black', label = 'Centroids')

plt.xlabel('Income')
plt.ylabel('Spending Score')
plt.legend()
plt.show()

Based on the plot, we see that the most valued customers  with high spending score are either low income or high income, but not middle income.

Next we see if age has an effect on the spending score

In [ ]:
# 10.
X = testdata.drop(columns=['genre','income'])
print(X.columns.values)
wcss=[]
for i in range(1,11):
    kmeans = KMeans(n_clusters=i,init='k-means++')
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
fig = plt.figure(figsize=(5,5))
plt.plot(np.arange(1,11),wcss)
plt.title('Elbow Curve')
plt.show() 

In [ ]:
# 11. Use the optimal k value from the plot
kmeans=KMeans(n_clusters=4,init='k-means++')
kmeans.fit(X)
y_kmeans=kmeans.predict(X)

In [ ]:
# 12. Plot the resulting clusters
fig = plt.figure(figsize=(5,5))

# the next 4 lines should be in a loop
plt.scatter(X[y_kmeans == 0].age, X[y_kmeans == 0].spend, label = 'Cluster 1')
plt.scatter(X[y_kmeans == 1].age, X[y_kmeans == 1].spend, label = 'Cluster 2')
plt.scatter(X[y_kmeans == 2].age, X[y_kmeans == 2].spend, label = 'Cluster 3')
plt.scatter(X[y_kmeans == 3].age, X[y_kmeans == 3].spend, label = 'Cluster 4')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c = 'black', label = 'Centroids')
plt.xlabel('Age')
plt.ylabel('Spending Score (1-100)')
plt.legend()
plt.show()

In [ ]:
# 13. What is the observation about the customers with high spending score?
# They are under 40

Now we check if the genre (or gender) has an effect on the spending score

In [ ]:
# 14. genre as a factor
X = testdata.drop(columns=['age','income'])
print(X.columns.values)
# change the genre to a number
genre = {'Male':0,'Female':1}
X = X.replace(genre)
# print the first lines of X to see the change
print(X.head())

# find the optimal k value
wcss=[]
for i in range(1,11):
    kmeans=KMeans(n_clusters=i,init='k-means++')
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
fig = plt.figure(figsize=(5,5))
plt.plot(np.arange(1,11),wcss)
plt.title('Elbow Curve')
plt.show() 

In [ ]:
# 15. use the optimal k value to train and test the model
kmeans=KMeans(n_clusters=3,init='k-means++')
kmeans.fit(X)
y_kmeans=kmeans.predict(X)

In [ ]:
# 16. plot the clusters
fig = plt.figure(figsize=(5,5))

# The next 3 lines should be in a loop:
plt.scatter(X[y_kmeans == 0].genre, X[y_kmeans == 0].spend, label = 'Cluster 1')
plt.scatter(X[y_kmeans == 1].genre, X[y_kmeans == 1].spend, label = 'Cluster 2')
plt.scatter(X[y_kmeans == 2].genre, X[y_kmeans == 2].spend, label = 'Cluster 3')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c = 'black', label = 'Centroids')
plt.xlabel('Genre')
plt.ylabel('Spending Score')
plt.legend()
plt.show()

In [ ]:
# 17. What can be observed about the genre's effect on spending score?
# genre doesn't matter

In [ ]:
# 18. Check to see if there's any correlation between income and age
X = testdata.drop(columns=['genre','spend'])
print(X.columns.values)
fig = plt.figure(figsize=(5,5))
plt.scatter(X.income, X.age)
plt.xlabel('Income')
plt.ylabel('Age')
plt.show()

In [ ]:
# 19. Is there correlation between age and income?
# no real correlation between age and income, except for the top income
# level, where people are between 30-50 years old

In [ ]:
# 20. What can the mall conclude about its shoppers?
# Conclusion: the most active customers are between 20-40 in age (younger)
# and are either in the low or high income, not middle income.

In [ ]:
# 21. Looking at both age and income
X = testdata.drop(columns=['genre'])
print(X.head())
wcss=[]
for i in range(1,11):
    kmeans=KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=1)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
fig = plt.figure(figsize=(5,5))
plt.plot(np.arange(1,11),wcss)
plt.title('Elbow curve')
plt.show() 

In [ ]:
# 21. 
kmeans=KMeans(n_clusters=5,init='k-means++')
kmeans.fit(X)
y_kmeans=kmeans.predict(X)

In [ ]:
# 22.
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')

# The next 5 lines should be in a loop
ax.scatter(X.age[y_kmeans == 0], X.income[y_kmeans == 0], X.spend[y_kmeans == 0], label = 'Cluster 1')
ax.scatter(X.age[y_kmeans == 1], X.income[y_kmeans == 1], X.spend[y_kmeans == 1], label = 'Cluster 2')
ax.scatter(X.age[y_kmeans == 2], X.income[y_kmeans == 2], X.spend[y_kmeans == 2], label = 'Cluster 3')
ax.scatter(X.age[y_kmeans == 3], X.income[y_kmeans == 3], X.spend[y_kmeans == 3], label = 'Cluster 4')
ax.scatter(X.age[y_kmeans == 4], X.income[y_kmeans == 4], X.spend[y_kmeans == 4], label = 'Cluster 5')
plt.xlabel('Age')
plt.ylabel('Income')
ax.set_zlabel('Spending Score')
ax.view_init(10,20)   # change these values to move / rotate the plot
plt.legend()
plt.show()

Map the cluster back to the customer data

In [ ]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = X.index.values
cluster_map['cluster'] = y_kmeans
cluster_map
# shows which cluster each row of the customer data is in

In [ ]:
# choosing cluster 4 only (one group of highest spending customers)
cluster_map[cluster_map.cluster == 3]

In [ ]:
# ind is index of rows that belong to cluster 4
ind = cluster_map[cluster_map.cluster == 3].data_index.values
data.loc[ind]
# using ind to find all customers that belong to cluster 1

The goal of k-means clustering is to group data into non-overlapping clusters. It is very useful when the clusters have a tight, spherical shape because the attributes of the data in the clusters are clear.

### Principal Component Analysis (PCA)

Unlike the dataset used in the k-means clustering example above, which has 5 features, often a dataset is made of many features. While a ML model can work in N-dimensional space where N is large, it can be computationally expensive and the output difficult to interpret. For example, it is easy to see the clusters in a 2D or 3D plot, but it would be  more complicated to work with and visualize a 10-dimensional plot. This is where principal component analysis (PCA) comes in.

PCA is used to reduce the dimensionality of large datasets, by transforming a large set of variables into a smaller one that still contains most of the information in the large set. Linear algebra concepts (covariance matrices and eigenvectors) are used to find the principal components of the data. The resulting principal components are uncorrelated (orthogonal), and most of the information from the original variables is compressed into the principal components.

PCA can be used with both supervised and unsupervised learning to simplify the dataset. But PCA can also be used as an unsupervised algorithm on its own. By reducing the large number of features into a few variables, it is easier to observe trends or clusters.

<br>Clustering and dimension reduction are two main unsupervised learning algorithms. These algorithms find hidden patterns or similarities and differences in datasets without needing output labels. Therefore they are good for exploratory data analysis, customer segmentation, and image recognition, which are areas where it's difficult to obtain the y output labels.